In [23]:
import json
import yaml
import os.path as op
from yaml.loader import SafeLoader

In [6]:
def read_yaml(path_to_config_file):
    """
    Input:
    the path to the config file

    Returns
    a dictionary that contains all the config info

    """
    with open(path_to_config_file, "r") as v:
        config = yaml.load(v, Loader=SafeLoader)

    """     container = config["general"]["container"]
        host = config["general"]["host"]
        njobs = config["host_options"][host]["njobs"]
        if njobs == "" or njobs is None:
            njobs = 2
        host_str = f"{host}"
        if host == "local":
            launch_mode = config["host_options"]["local"]["launch_mode"]
            valid_options = ["serial", "parallel","dask_worker"]
            if launch_mode in valid_options:
                host_str = (
                    f"{host_str}, and commands will be launched in {launch_mode} mode "
                    f"every {njobs} jobs. "
                    f"Serial is safe but it will take longer. "
                    f"If you launch in parallel be aware that some of the "
                    f"processes might be killed if the limit (usually memory) "
                    f"of the machine is reached. "
                )
            else:
                die(
                    f"local:launch_mode {launch_mode} was passed, valid options are {valid_options}"
                )

        logger.warning(
            "\n"
            + "#####################################################\n"
            + f"Successfully read the config file {path_to_config_file} \n"
            + f'Basedir is: {config["general"]["basedir"]} \n'
            + f'Container is: {container}_{config["container_specific"][container]["version"]} \n'
            + f"Host is: {host_str} \n"
            + f'analysis folder is: {config["general"]["analysis_name"]} \n'
            + f"##################################################### \n"
        )
    """
    return config

In [8]:
lc_config=read_yaml('/Users/tiger/soft/launchcontainers/example_configs/0.3.0/example_config.yaml')
container="freesurferator"

In [31]:
yaml_info=lc_config["container_specific"][container]
fs_json_keys=['pre_fs','control_points','annotfile', 'mniroizip']
fs_json_val=['pre_fs/existingFS.zip','control_points/control.dat','mniroizip/mniroizip.zip','annotfile/annotfile.zip']

fs_json_dict= {key: value for key, value in zip(fs_json_keys, fs_json_val)}
json_template={'anat': 
                    {'location': {
                        'path': '/flywheel/v0/input/anat/T1.nii.gz', 
                        'name': 'T1.nii.gz',
                     },
                    'base': 'file'}
                    }

In [36]:
for key in fs_json_dict.keys():
    if key in yaml_info.keys() and yaml_info[key]:
        json_template[key] = {
                'location': {
                    'path': op.join('/flywheel/v0/input', fs_json_dict[key]), 
                    'name': op.basename(fs_json_dict[key])
                },
                'base': 'file'
            }
    if 'anat' in json_template.keys() and 'pre_fs' in json_template.keys():
        json_template.pop('anat')

In [37]:
json_template

{'pre_fs': {'location': {'path': '/flywheel/v0/input/pre_fs/existingFS.zip',
   'name': 'existingFS.zip'},
  'base': 'file'},
 'control_points': {'location': {'path': '/flywheel/v0/input/control_points/control.dat',
   'name': 'control.dat'},
  'base': 'file'}}

In [17]:
fs_json_dict

{'pre_fs_True': 'pre_fs/existingFS.zip',
 'pre_fs_False': 'anat/T1.nii.gz',
 'control_points': 'control_points/control.dat',
 'annotfile': 'mniroizip/mniroizip.zip',
 'mniroizip': 'annotfile/annotfile.zip'}

In [20]:
fs_json_dict.keys()

dict_keys(['pre_fs_True', 'pre_fs_False', 'control_points', 'annotfile', 'mniroizip'])

In [22]:
yaml_info.keys()

dict_keys(['version', 'pre_fs', 'control_points', 'source_path_fszip', 'precontainer_anat', 'anat_analysis_name', 'precontainer_fmriprep', 'fmriprep_analysis_name', 'prefs_zipname', 'prefs_unzipname', 'annotfile', 'mniroizip'])

In [40]:
preproc_json_keys=['ANAT','BVAL','BVEC', 'DIFF','FSMASK']
preproc_json_val=['ANAT/T1.nii.gz','BVAL/dwiF.bval','BVEC/dwiF.bvec','DIFF/dwiF.nii.gz','FSMASK/brainmask.nii.gz']


In [59]:
container="rtp2-pipeline"
pipeline_json_keys=['anatomical','bval','bvec', 'dwi','fs','tractparams']
pipeline_json_val=['anatomical/T1.nii.gz','bval/dwi.bval','bvec/dwi.bvec','dwi/dwi.nii.gz','fs/fs.zip','tractparams/tractparams.csv']


In [65]:
def get_config_dict(container,lc_config,rtp2_json_keys,rtp2_json_val):
    config_info_dict = {}
    yaml_info=lc_config["container_specific"][container]
    
    rtp2_json_dict= {key: value for key, value in zip(rtp2_json_keys, rtp2_json_val)}

    
    if container == "freesurferator":
        config_json_extra={'anat': 
                    {'location': {
                        'path': '/flywheel/v0/input/anat/T1.nii.gz', 
                        'name': 'T1.nii.gz',
                    },
                    'base': 'file'}
                    }
        for key in rtp2_json_dict.keys():
            if key in yaml_info.keys() and yaml_info[key]:
                config_json_extra[key] = {
                        'location': {
                            'path': op.join('/flywheel/v0/input', rtp2_json_dict[key]), 
                            'name': op.basename(rtp2_json_dict[key])
                        },
                        'base': 'file'
                    }
            if 'anat' in config_json_extra.keys() and 'pre_fs' in config_json_extra.keys():
                config_json_extra.pop('anat')

    else:
        config_json_extra={}
        for key in rtp2_json_dict.keys():
            config_json_extra[key] = {
                    'location': {
                        'path': op.join('/flywheel/v0/input', rtp2_json_dict[key]), 
                        'name': op.basename(rtp2_json_dict[key])
                    },
                    'base': 'file'
                }
        
    config_info_dict['input']= config_json_extra   
    return config_info_dict

In [66]:
config_info_dict=get_config_dict(container,lc_config,pipeline_json_keys,pipeline_json_val)

In [67]:
config_info_dict

{'input': {'anatomical': {'location': {'path': '/flywheel/v0/input/anatomical/T1.nii.gz',
    'name': 'T1.nii.gz'},
   'base': 'file'},
  'bval': {'location': {'path': '/flywheel/v0/input/bval/dwi.bval',
    'name': 'dwi.bval'},
   'base': 'file'},
  'bvec': {'location': {'path': '/flywheel/v0/input/bvec/dwi.bvec',
    'name': 'dwi.bvec'},
   'base': 'file'},
  'dwi': {'location': {'path': '/flywheel/v0/input/dwi/dwi.nii.gz',
    'name': 'dwi.nii.gz'},
   'base': 'file'},
  'fs': {'location': {'path': '/flywheel/v0/input/fs/fs.zip',
    'name': 'fs.zip'},
   'base': 'file'},
  'tractparams': {'location': {'path': '/flywheel/v0/input/tractparams/tractparams.csv',
    'name': 'tractparams.csv'},
   'base': 'file'}}}

In [71]:
config_info_dict['cc']

KeyError: 'cc'

In [74]:
if "input" in config_info_dict:
    print("in")

in
